In [ ]:
from bs4 import BeautifulSoup as bs
import glob, os, re, csv
from datetime import datetime

In [ ]:
startTime = datetime.now()

In [ ]:
# parse for applicants

def parse(xmlfile):

    global xmlnum
    # numbers in the function names represents how deep in the tree teh function needs to go
    # and if it needs to get an attribute of that tag or parse numbers

    def iferror(something):

        try:
            something
        except AttributeError:
            return ""


    def iferror1(tagname):

        try:
            return soup.find(tagname).text
        except AttributeError:
            return ""

    def iferror1get(tagname,attrib):

        try:
            return soup.find(tagname).get(attrib)
        except AttributeError:
            return ""
     
    def iferror1attrib(tagname,attrib,value):

        try:
            return soup.find(tagname, attrs={attrib: value}).text
        except AttributeError:
            return ""
    


    def iferror2(tagname,tagname2):

        try:
            return soup.find(tagname).find(tagname2).text
        except AttributeError:
            return ""

    def iferror2num(tagname,tagname2):

        try:
            return float(soup.find(tagname).find(tagname2).text.replace(",",".").replace(" ",""))
        except (AttributeError, ValueError):
            return ""

    def iferror2get(tagname,tagname2,attrib):

        try:
            return soup.find(tagname).find(tagname2).get(attrib)
        except AttributeError:
            return ""

    def makedate(expression):
        
        try:
            pd = datetime.strptime(expression, "%Y%m%d").date()
            return pd.strftime('%Y-%m-%d')
        except:
            return ''


    soup = bs(xmlfile)

    #publication attrs
    ID = iferror1get("bibliographic-data","id")
    # print ID
    url = re.sub("P$","","https://register.epo.org/application?number="+ID)
    title = iferror1attrib("invention-title", "lang", "en")
    status = iferror1get("bibliographic-data","status")
    public_date = makedate(iferror2("publication-reference","date"))
    public_number = iferror2("publication-reference","doc-number")
    appl_date = makedate(iferror2("application-reference","date"))
    appl_number = iferror2("application-reference","doc-number")
    event_date = makedate(iferror2("events-data","date")) #most recent
    event_descr = iferror2("events-data","event-text")

    #other applicants in the patent?
    if soup.find("applicant", {"sequence" : "2"}) != None:
        others = "Y"
    else:
        others = "N"

    #citations
    citations = []

    try:
        for cit in soup.find("references-cited").findAll("citation"):
        
            try:
                cit_nr = cit.find("doc-number").text
            except AttributeError:
                cit_nr = cit.get("id")
 
            citations.append(cit_nr)

            citations_num = len(citations)
            citations_nrs = ";".join(citations)
 
    except AttributeError:
     
        citations_num = 0           

    #oponents
    if soup.find("opponent") != None:
        oponent = "Y"
    else:
        oponent = "N"

    lapses = []
    lapses_nr = None
    if soup.find("term-of-grant") != None:
        for lapse in soup.find("term-of-grant").findAll("lapsed-in-country"):
            lapses.append(lapse.find("date").text)
        
        lapses_nr = len(lapses)

    row = []
    
    def get_countries(tagname):
        
        country_list = []
        
        if soup.find(tagname+'s') != None:
            for ca in soup.find(tagname+"s").find_all(tagname):
                    
                country_list.append(ca.find("country").text)
                    
        return ";".join(country_list)
            
    inventors_countries = get_countries("inventor")
    representatives_countries = get_countries("agent")
    applicants_countries = get_countries("applicant")
    
    if soup.find("applicants") != None:

        for ca in soup.find("applicants").find_all("applicant"):

            contract = []    
            soup = ca
            naam = iferror1("name")
            straat = iferror1("address-1")
            pscenstad = iferror1("address-2")
            land = iferror1("country")

            if naam == '':
                continue
            else: 
                contract.extend([ID,url,title,status,public_date,public_number,naam,straat,pscenstad,land,applicants_countries,inventors_countries,representatives_countries,appl_date,appl_number,event_date,event_descr,citations_num,oponent,lapses_nr])
                row.append(contract)

    else:
        contract = []
        naam = "geen"
        straat = "geen"
        pscenstad = "geen"
        land = "geen"

        contract.extend([ID,url,title,status,public_date,public_number,naam,straat,pscenstad,land,applicants_countries,inventors_countries,representatives_countries,appl_date,appl_number,event_date,event_descr,citations_num,oponent,lapses_nr])
        row.append(contract)

    #print(row)
    return row

In [ ]:
directory = '../'
files = glob.glob(directory+'*.xml')

in case of hiccups

In [ ]:
parsed = []

In [ ]:
new = [f for f in files if f not in parsed]

In [ ]:
with open("../patents-example.csv", "w",  newline='') as outputfile:
    writer = csv.writer(outputfile)
    header = ["ID","url","title","status","publication_date","publication_number",
              "applicant_name","applicant_street","applicant_city","applicant_land",
              "applicants_from","inventors_from","representatives_from",
              "application_date","application_number","last_event_date","last_event_description",
              "number_of_citations","oponents","nr_of_lapses"]

    writer.writerow(header)
    
    for f in new:

        filename = open(f).read()
        row = parse(filename)
        
        if row == None:
            continue
        else:
            writer.writerows(row)
            
        parsed.append(f)